# Fairify Artifact
Artifact of the paper "Fairify: Fairness Verification of Neural Networks" from ICSE 2023.

To access ChameleonCloud resources, you may need the account to log in to ChameleonCloud. You also need to have a project to allocate resources (e.g., node).

In [1]:
from chi import context

context.version = "1.0"

context.choose_site(default="CHI@TACC")
context.choose_project()

### Check available hardware

In [2]:
from chi import hardware

node_type = "gpu_p100"
available_nodes = hardware.get_nodes(node_type=node_type, filter_reserved=True)
if available_nodes:
    print(f"There currently are {len(available_nodes)} {node_type} nodes ready to use")
else:
    print(f"All {node_type} nodes are in use! You could use next_free_timeslot to see how long you need to wait, or use the calendar.")

There currently are 2 gpu_p100 nodes ready to use


### Reserve node

In [3]:
from chi import lease
from datetime import timedelta
import os

my_lease = lease.Lease(f"{os.getenv('USER')}-power-management", duration=timedelta(hours=3))
my_lease.add_node_reservation(nodes=[available_nodes[0]]) # or you could use node_type=node_type
my_lease.add_fip_reservation(1) # include a floating ip
my_lease.submit(idempotent=True)

Waiting for lease to start... This can take up to 60 seconds


Lease radhofanazizi_gmail_com-power-management has reached status active


### Create a server on the node

In [4]:
from chi import server

my_server = server.Server(
    f"{os.getenv('USER')}-power-management",
    reservation_id=my_lease.node_reservations[0]["id"],
    image_name="CC-Ubuntu22.04", # or use image_name
)
my_server.submit(idempotent=True)

Waiting for server radhofanazizi_gmail_com-power-management's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,radhofanazizi_gmail_com-power-management
Id,9b4ea2f8-e1cd-4fe3-b0b3-90d512ae85e3
Status,ACTIVE
Image Name,CC-Ubuntu22.04
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.3.247 (v4) Type: fixed MAC: 44:a8:42:26:3f:4f
Network Name,sharednet1
Created At,2025-03-31T06:16:30Z
Keypair,trovi-b30b811
Reservation Id,2b6fb9c7-34eb-4c37-9f5c-74cfbff85614
Host Id,b281b13a05d4a4d342f673906de4005142c2819a049809e34ac97306


### Configure networking on the node

In [5]:
fip = my_lease.get_reserved_floating_ips()[0]
my_server.associate_floating_ip(fip)
my_server.check_connectivity(host=fip)

Checking connectivity to 129.114.109.67 port 22.


Connection successful


### Setup nvidia drivers and cuda

Download drivers

In [18]:
my_server.execute("""
sudo apt-get --purge remove -y '*cublas*' 'cuda*' 'nvidia-*' && \
sudo apt-get autoremove -y && \
sudo apt-get autoclean && \
sudo rm -rf /usr/local/cuda* && \
sudo apt update && \
sudo apt install -y nvidia-driver-470 && \
wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-keyring_1.0-1_all.deb && \
sudo dpkg -i cuda-keyring_1.0-1_all.deb && \
sudo apt update && \
sudo apt install -y cuda-toolkit-11-2
""")

Reading package lists...
Building dependency tree...
Reading state information...
Package 'libcublaslt.so.11' is not installed, so not removed
Package 'cuda-license-10-1' is not installed, so not removed
Package 'cuda-license-10-2' is not installed, so not removed
Package 'cuda-cudart-cross-amd64-11-6' is not installed, so not removed
Package 'cuda-cudart-cross-amd64-11-8' is not installed, so not removed
Package 'cuda-nvcc-11-9' is not installed, so not removed
Package 'cuda-cudart-cross-amd64-12-0' is not installed, so not removed
Package 'cuda-cudart-cross-amd64-12-4' is not installed, so not removed
Package 'cuda-nvcc-12-7' is not installed, so not removed
Package 'cuda-nvcc-12-9' is not installed, so not removed
Package 'cuda-drivers-diagnostic' is not installed, so not removed
Package 'cuda-cudart-cross-amd64-12-5' is not installed, so not removed
Package 'cuda-drivers-grid' is not installed, so not removed
Package 'cuda-cudart-cross-amd64-12-6' is not installed, so not removed
P

Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu jammy InRelease [270 kB]
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 270 kB in 1s (421 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
112 packages can be upgraded. Run 'apt list --upgradable' to see them.


Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libdrm-amdgpu1 libdrm-intel1 libdrm-nouveau2 libdrm-radeon1 libegl-mesa0
  libegl1 libfontenc1 libgl1 libgl1-mesa-dri libglapi-mesa libglvnd0
  libglx-mesa0 libglx0 libice6 libllvm15 libnvidia-cfg1-470
  libnvidia-common-470 libnvidia-compute-470 libnvidia-decode-470
  libnvidia-egl-wayland1 libnvidia-encode-470 libnvidia-extra-470
  libnvidia-fbc1-470 libnvidia-gl-470 libnvidia-ifr1-470 libpciaccess0
  libsensors-config libsensors5 libsm6 libwayland-client0 libx11-xcb1 libxaw7
  libxcb-dri2-0 libxcb-dri3-0 libxcb-glx0 libxcb-present0 libxcb-shm0
  libxcb-sync1 libxcb-xfixes0 libxcvt0 libxfixes3 libxfont2 libxkbfile1
  libxmu6 libxpm4 libxshmfence1 libxt6 libxxf86vm1 nvidia-compute-utils-470
  nvidia-dkms-470 nvidia-kernel-common-470 nvidia-kernel-source-470
  nvidia-utils-470 x11-common x11-xkb-utils xserver-common xserver-xorg-core
  xserver-xorg-vi

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 272 MB in 7s (36.4 MB/s)
Selecting previously unselected package libdrm-amdgpu1:amd64.
(Reading database ... 84390 files and directories currently installed.)
Preparing to unpack .../00-libdrm-amdgpu1_2.4.113-2~ubuntu0.22.04.1_amd64.deb ...
Unpacking libdrm-amdgpu1:amd64 (2.4.113-2~ubuntu0.22.04.1) ...
Selecting previously unselected package libpciaccess0:amd64.
Preparing to unpack .../01-libpciaccess0_0.16-3_amd64.deb ...
Unpacking libpciaccess0:amd64 (0.16-3) ...
Selecting previously unselected package libdrm-intel1:amd64.
Preparing to unpack .../02-libdrm-intel1_2.4.113-2~ubuntu0.22.04.1_amd64.deb ...
Unpacking libdrm-intel1:amd64 (2.4.113-2~ubuntu0.22.04.1) ...
Selecting previously unselected package libdrm-nouveau2:amd64.
Preparing to unpack .../03-libdrm-nouveau2_2.4.113-2~ubuntu0.22.04.1_amd64.deb ...
Unpacking libdrm-nouveau2:amd64 (2.4.113-2~ubuntu0.22.04.1) ...
Selecting previously unselected package libdrm-radeon1:amd64.
Preparing to unpack .../04-libdrm-radeon1_2.4.

...

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu jammy InRelease [270 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1581 B]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [2068 kB]
Fetched 2595 kB in 1s (2126 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
130 packages can be upgraded. Run 'apt list --upgradable' to see them.


Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  ca-certificates-java cuda-command-line-tools-11-2 cuda-compiler-11-2
  cuda-cudart-11-2 cuda-cudart-dev-11-2 cuda-cuobjdump-11-2 cuda-cupti-11-2
  cuda-cupti-dev-11-2 cuda-cuxxfilt-11-2 cuda-documentation-11-2
  cuda-driver-dev-11-2 cuda-gdb-11-2 cuda-libraries-11-2
  cuda-libraries-dev-11-2 cuda-memcheck-11-2 cuda-nsight-11-2
  cuda-nsight-compute-11-2 cuda-nsight-systems-11-2 cuda-nvcc-11-2
  cuda-nvdisasm-11-2 cuda-nvml-dev-11-2 cuda-nvprof-11-2 cuda-nvprune-11-2
  cuda-nvrtc-11-2 cuda-nvrtc-dev-11-2 cuda-nvtx-11-2 cuda-nvvp-11-2
  cuda-samples-11-2 cuda-sanitizer-11-2 cuda-tools-11-2 cuda-visual-tools-11-2
  default-jre default-jre-headless fontconfig-config fonts-dejavu-core
  java-common libavahi-client3 libavahi-common-data libavahi-common3 libcairo2
  libcublas-11-2 libcublas-dev-11-2 libcufft-11-2 libcufft-dev-11-2 libcups2
  libcurand-11-2 l

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 2144 MB in 1min 54s (18.7 MB/s)
Selecting previously unselected package java-common.
(Reading database ... 85329 files and directories currently installed.)
Preparing to unpack .../00-java-common_0.72build2_all.deb ...
Unpacking java-common (0.72build2) ...
Selecting previously unselected package libavahi-common-data:amd64.
Preparing to unpack .../01-libavahi-common-data_0.8-5ubuntu5.2_amd64.deb ...
Unpacking libavahi-common-data:amd64 (0.8-5ubuntu5.2) ...
Selecting previously unselected package libavahi-common3:amd64.
Preparing to unpack .../02-libavahi-common3_0.8-5ubuntu5.2_amd64.deb ...
Unpacking libavahi-common3:amd64 (0.8-5ubuntu5.2) ...
Selecting previously unselected package libavahi-client3:amd64.
Preparing to unpack .../03-libavahi-client3_0.8-5ubuntu5.2_amd64.deb ...
Unpacking libavahi-client3:amd64 (0.8-5ubuntu5.2) ...
Selecting previously unselected package libcups2:amd64.
Preparing to unpack .../04-libcups2_2.4.1op1-1ubuntu4.11_amd64.deb ...
Unpacking libcups2:amd

<Result cmd="\nsudo apt-get --purge remove -y '*cublas*' 'cuda*' 'nvidia-*' && sudo apt-get autoremove -y && sudo apt-get autoclean && sudo rm -rf /usr/local/cuda* && sudo apt update && sudo apt install -y nvidia-driver-470 && wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-keyring_1.0-1_all.deb && sudo dpkg -i cuda-keyring_1.0-1_all.deb && sudo apt update && sudo apt install -y cuda-toolkit-11-2\n" exited=0>

Restart nvidia services

In [19]:
my_server.execute("sudo modprobe -r nouveau")
my_server.execute("sudo rmmod nouveau 2>/dev/null || true")
my_server.execute("sudo update-initramfs -u")
my_server.execute("sudo modprobe nvidia")

update-initramfs: Generating /boot/initrd.img-5.15.0-124-generic


<Result cmd='sudo modprobe nvidia' exited=0>

Verifiy services

In [21]:
my_server.execute("nvidia-smi")
my_server.execute("""
if [ -L /usr/local/bin/nvcc ]; then sudo rm /usr/local/bin/nvcc; fi && \
sudo ln -s /usr/local/cuda-11.2/bin/nvcc /usr/local/bin/nvcc
""")
my_server.execute("nvcc --version")

Mon Mar 31 07:13:54 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.256.02   Driver Version: 470.256.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:03:00.0 Off |                    0 |
| N/A   28C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:82:00.0 Off |                    0 |
| N/A   

<Result cmd='nvcc --version' exited=0>

## Run Fairify

Now, we can finally run Fairify. First we need to clone the github repo first and then run the reprduce.sh script which contain the instructions from README.md
packaged into a bash file

In [22]:
my_server.execute("rm -rf Fairify && git clone https://github.com/radhofan/Fairify.git")

Cloning into 'Fairify'...


<Result cmd='rm -rf Fairify && git clone https://github.com/radhofan/Fairify.git' exited=0>

### Run dependencies and first experiment

In [23]:
my_server.execute("chmod +x Fairify/reproduce.sh")
my_server.execute("bash Fairify/reproduce.sh")

ERROR: File or directory already exists: '/home/cc/miniconda'
If you want to update an existing installation, use the -u option.


Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: ...working... done

# All requested packages already installed.

Running `shell init`, which:
 - modifies RC file: "/home/cc/.bashrc"
 - generates config for root prefix: "/home/cc/miniconda"
 - sets mamba executable to: "/home/cc/miniconda/bin/mamba"
The following has been added in your "/home/cc/.bashrc" file

# >>> mamba initialize >>>
# !! Contents within this block are managed by 'mamba shell init' !!
export MAMBA_EXE='/home/cc/miniconda/bin/mamba';
export MAMBA_ROOT_PREFIX='/home/cc/miniconda';
__mamba_setup="$("$MAMBA_EXE" shell hook --shell bash --root-prefix "$MAMBA_ROOT_PREFIX" 2> /dev/null)"
if [ $? -eq 0 ]; then
    eval "$__mamba_setup"
else
    alias mamba="$MAMBA_EXE"  # Fallback on help from mamba activate
fi
unset __mamba_setup
# <<< mamba initialize <<<



warning  libmamba 'repo.anaconda.com', a commercial channel hosted by Anaconda.com, is used.
    
warning  libmamba Please make sure you understand Anaconda Terms of Services.
    
warning  libmamba See: https://legal.anaconda.com/policies/en/




Transaction

  Prefix: /home/cc/miniconda/envs/fairify

  Updating specs:

   - python=3.9


  Package               Version  Build           Channel         Size
───────────────────────────────────────────────────────────────────────
  Install:
───────────────────────────────────────────────────────────────────────

  + _libgcc_mutex           0.1  main            pkgs/main     Cached
  + _openmp_mutex           5.1  1_gnu           pkgs/main     Cached
  + ca-certificates   2025.2.25  h06a4308_0      pkgs/main     Cached
  + ld_impl_linux-64       2.40  h12ee557_0      pkgs/main     Cached
  + libffi                3.4.4  h6a678d5_1      pkgs/main     Cached
  + libgcc-ng            11.2.0  h1234567_1      pkgs/main     Cached
  + libgomp              11.2.0  h1234567_1      pkgs/main     Cached
  + libstdcxx-ng         11.2.0  h1234567_1      pkgs/main     Cached
  + ncurses                 6.4  h6a678d5_0      pkgs/main     Cached
  + openssl              3.0.16  h5eee18b_0      

2025-03-31 07:15:29.110197: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


...


Processing Partitions:   0%|          | 0/201 [00:00<?, ?it/s]

Pruning done!
Verifying ...
sat
V time:  75.68
['0', '0', '0', '1', '7227', '0', '0', '2', '0', '4', '2', '0', '0', '0', '1', '1', '1', '0', '0', '0']
['0', '0', '0', '1', '7227', '0', '0', '2', '0', '4', '2', '1', '0', '0', '1', '1', '1', '0', '0', '0']
[-0.14900333] [0.04331559]
[-0.14900333] [0.04331571]
******************
INTERVAL BASED PRUNING
SINGULAR VERIFICATION
Pruning done!
Verifying ...
unknown
Pruning done!
11.76 % HEURISTIC PRUNING
66.67 % TOTAL PRUNING
Verifying ...
unknown
V time:  100.067
******************
INTERVAL BASED PRUNING
SINGULAR VERIFICATION
Pruning done!
Verifying ...
unknown
Pruning done!
9.8 % HEURISTIC PRUNING
82.35 % TOTAL PRUNING
Verifying ...
sat
V time:  2.287
['0', '3', '0', '0', '6003', '1', '0', '3', '0', '3', '1', '1', '1', '0', '2', '0', '2', '0', '0', '0']
['0', '3', '0', '0', '6003', '1', '0', '3', '0', '3', '1', '0', '1', '0', '2', '0', '2', '0', '0', '0']
[0.19230536] [-1.3530254e-05]
[0.19230548] [-1.3411045e-05]
******************
INTERVAL B


Processing Models: 100%|██████████| 5/5 [1:42:04<00:00, 1224.88s/it]


INTERVAL BASED PRUNING
SINGULAR VERIFICATION
Pruning done!
Verifying ...
sat
V time:  97.089
['0', '0', '0', '0', '3607', '0', '0', '4', '0', '1', '2', '0', '0', '0', '1', '0', '1', '0', '0', '0']
['0', '0', '0', '0', '3607', '0', '0', '4', '0', '1', '2', '1', '0', '0', '1', '0', '1', '0', '0', '0']
[-0.09856033] [0.0612945]
[-0.09856033] [0.06129456]
******************
INTERVAL BASED PRUNING
SINGULAR VERIFICATION
Pruning done!
Verifying ...
unknown
Pruning done!
3.96 % HEURISTIC PRUNING
87.13 % TOTAL PRUNING
Verifying ...
sat
V time:  26.34
['1', '0', '0', '0', '15600', '0', '1', '1', '1', '1', '0', '1', '2', '0', '1', '0', '1', '1', '1', '0']
['1', '0', '0', '0', '15600', '0', '1', '1', '1', '1', '0', '0', '2', '0', '1', '0', '1', '1', '1', '0']
[0.09613389] [-0.05688226]
[0.09613401] [-0.0568822]
******************
INTERVAL BASED PRUNING
SINGULAR VERIFICATION
Pruning done!
Verifying ...
unknown
Pruning done!
2.97 % HEURISTIC PRUNING
83.17 % TOTAL PRUNING
Verifying ...
unknown
V time

<Result cmd='bash Fairify/reproduce.sh' exited=0>

### Run second experiment

In [ ]:
my_server.execute("bash Fairify/stress/fairify-stress.sh Fairify/stress/GC/Verify-GC.py")

### Run third experiment

In [ ]:
my_server.execute("bash Fairify/relaxed/fairify-relaxed.sh Fairify/relaxed/GC/Verify-GC.py")

### Run fourth experiment

In [ ]:
my_server.execute("bash Fairify/targeted/fairify-targeted.sh Fairify/targeted/GC/Verify-GC.py")